In [1]:
import pandas as pd
import tokenize_uk
from typing import List
from ast import literal_eval

In [2]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

In [3]:
def read_all_frames(files):
    frames= []
    for file in files:
        df = pd.read_csv(file)
        frames.append(df)
    
    return pd.concat(frames)

In [4]:
files = [
    'dataset/booking/booking-train.csv',
    'dataset/booking/booking-test.csv',
    'dataset/booking/booking-val.csv'
]

In [5]:
full_df = read_all_frames(files)

In [6]:
full_df

,title,pos_text,neg_text,ratingValue,bestRating,hotel,rating
0,Лише дівчата на рецепції - три рази мені мінял...,Лише дівчата на рецепції - три рази мені мінял...,"Все. Одного досвіду вистарчило, щоб більше сюд...",4.6,10.0,verhovina.uk.html,2
1,"Оформлення кімнати хороше, досить приємне, на ...","Оформлення кімнати хороше, досить приємне, на ...",Nan,9.2,10.0,verhovina.uk.html,5
2,"Усе відмінно, завдяки якісному сервісу ми завж...","Усе відмінно, завдяки якісному сервісу ми завж...",Рекомендую людям котрі подорожують власним тра...,10.0,10.0,verhovina.uk.html,5
3,Ціна/якість в принципі,Ціна/якість в принципі,"душова, трохи старий ремонт, але за 500 грн за...",7.5,10.0,verhovina.uk.html,4
4,"Приїхали з сином біля сьомої ранку, сонні та в...","Приїхали з сином біля сьомої ранку, сонні та в...",Nan,8.3,10.0,verhovina.uk.html,4
...,...,...,...,...,...,...,...
30499,"Сауна, басейн, стоянка, обслуговування, ресторан","Сауна, басейн, стоянка, обслуговування, ресторан",Nan,10.0,10.0,izumrud.uk.html,5
30500,"Басейн, кухня в ресторані","Басейн, кухня в ресторані",Безкоштовний Wi-Fi не працює,9.2,10.0,izumrud.uk.html,5
30501,"Зручне місце розташування, парковка, комфортни...","Зручне місце розташування, парковка, комфортни...",Не працював басейн,8.8,10.0,izumrud.uk.html,4
30502,"Рівень хорошого готелю: чисто, зручно, безкошт...","Рівень хорошого готелю: чисто, зручно, безкошт...",Вайда працював погано.,8.3,10.0,izumrud.uk.html,4


In [7]:
def is_review_valid(review):
    if type(review) is not str:
        return False
    try:
        if len(review) == 0:
            return False
        if review == 'Nan':
            return False
        return True
    except:
        return False

In [8]:
def get_pos_neg_reviews(df):
    pos_texts = df['pos_text'].values
    neg_texts = df['neg_text'].values
    
    positives, negatives = [], []
    for i in range(0, len(df)):
        if is_review_valid(pos_texts[i]):
            positives.append(pos_texts[i])
            
        if is_review_valid(neg_texts[i]):
            negatives.append(neg_texts[i])
            
    return positives, negatives

In [9]:
pos_texts, neg_texts = get_pos_neg_reviews(full_df)

In [10]:
stop_words = read_stop_words('data/ukrainian-stopwords.txt')

In [11]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [12]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [13]:
!ls dataset/trip-advisor/

analyze.ipynb
create-text-classification-dataset.ipynb
tripadvisor-dnipro.csv
tripadvisor-ivano_frankivsk.csv
tripadvisor-kharkiv.csv
tripadvisor-kyiv.csv
tripadvisor-lviv.csv
tripadvisor-odesa.csv
trip-advisor-test.csv
trip-advisor-text-classification-test.csv
trip-advisor-text-classification-train.csv
trip-advisor-text-classification-val.csv
trip-advisor-train.csv
tripadvisor-uzhhorod.csv
trip-advisor-val.csv


In [114]:
trip_advisor_df = pd.read_csv("dataset/trip-advisor/trip-advisor-val.csv")

In [110]:
blum = 'https://www.tripadvisor.ru/Hotel_Review-g295377-d14044085-Reviews-Blum_Hotel-Lviv_Lviv_Oblast.html'

In [115]:
trip_advisor_df.loc[trip_advisor_df['hotel'] == blum]

,hotel,review_ru,review_uk,ratings


In [17]:
booking_df = pd.read_csv('dataset/booking/booking-test.csv')

In [33]:
blum_df = full_df.loc[full_df['hotel'] == 'blum.uk.html']

### Neagtive TF-IDF

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
len(neg_texts)

96242

In [70]:
neg_cv=CountVectorizer(max_df=0.85,stop_words=stop_words,max_features=1000, ngram_range=(2,2))
neg_word_count_vector=neg_cv.fit_transform(full_df['neg_text'].values)
neg_word_count_vector.shape

/home/dbabenko/.local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(134083, 1000)

In [71]:
from sklearn.feature_extraction.text import TfidfTransformer

neg_tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
neg_tfidf_transformer.fit(neg_word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [18]:
# get the document that we want to extract keywords from
#doc="Хороше розташування. Безпечна парковка на території готелю. Наявність холодильника та чайника в номері.\nЩоденне прибирання."
#doc  = "Готель дуже зручно розташований. Якщо вам в номері не сидіти то можна спокійно бронювати. Сніданок хороший. Номери стандартні НЕ великі. Персонал ввічливий. Чисто. Після ремонту свіжо. Проблема була з водою, гаряча швидко закінчилася. Але на наступний день з'явилася)). Тапки були одні речі. Мабуть комплектацією дійсно не заморочуються. Але якщо б сказали принесли напевно, ми не просили. Вид з нашого Номери 305 був на вікна житлового будинку, але якщо вийти на балкончик і подивитися на право красиво. Відмінний вид з ресторану на місто. Таксі нам викликали на прощання, доїхали до вокзалу за 3 ціни))))) але зате водій був ввічливий і машина хороша. На фото вид з ресторану."
# doc = neg_texts[123]


#generate tf-idf for the given document
neg_tf_idf_vector=neg_tfidf_transformer.transform(neg_cv.transform(neg_texts))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(neg_tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(neg_cv.get_feature_names(),sorted_items,100)

# now print the results
# print("\n===Review text===")
# print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


===Keywords===
їжа ресторані 1.0
іншому номері 1.0
іншому місці 1.0
іншому готелі 1.0
іншого боку 1.0


### Positive TF-IDF

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [20]:
len(pos_texts)

120432

In [21]:
pos_cv=CountVectorizer(max_df=0.85,stop_words=stop_words,max_features=1000, ngram_range=(2,2))
pos_word_count_vector=pos_cv.fit_transform(full_df['pos_text'].values)
pos_word_count_vector.shape

/home/dbabenko/.local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(134083, 1000)

In [22]:
len(pos_cv.vocabulary_)

1000

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer

pos_tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
pos_tfidf_transformer.fit(pos_word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [24]:
pos_word_count_vector

<134083x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 298443 stored elements in Compressed Sparse Row format>

In [25]:
# you only needs to do this once
pos_feature_names=pos_cv.get_feature_names()

In [26]:
pos_feature_names

['10 10',
 '10 15',
 '10 хв',
 '10 хвилин',
 '10 хвилинах',
 '12 00',
 '14 00',
 '15 20',
 '15 хв',
 '15 хвилин',
 '15 хвилинах',
 '20 хв',
 '20 хвилин',
 '23 00',
 '30 хвилин',
 '39 лякливий',
 '39 язаний',
 '39 язка',
 '39 язково',
 '39 язку',
 '39 язно',
 '39 яке',
 '39 яти',
 '39 ятки',
 '39 яток',
 '39 ять',
 '39 єр',
 '39 єру',
 '39 єю',
 '39 їзд',
 '39 їзду',
 'quota quota',
 'wi fi',
 'адекватна ціна',
 'аеропорту жуляни',
 'акуратний номер',
 'безкоштовна парковка',
 'березі моря',
 'близьке розташування',
 'близькість метро',
 'близькість моря',
 'близькість центру',
 'білизна рушники',
 'білизна чиста',
 'білосніжна постіль',
 'білосніжні рушники',
 'біля вокзалу',
 'біля входу',
 'біля готелю',
 'біля метро',
 'біля моря',
 'вай фай',
 'ванна кімната',
 'ванною кімнатою',
 'ванні приналежності',
 'ванній кімнаті',
 'вартість номера',
 'вартість проживання',
 'ввічливий персонал',
 'ввічливий привітний',
 'ввічливий уважний',
 'ввічливий чуйний',
 'вдале розташування',
 'вел

In [60]:
def analyze_top_n_pos_bigram(positive_texts):
    pos_tf_idf_vector=pos_tfidf_transformer.transform(pos_cv.transform(positive_texts))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(pos_tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(pos_feature_names,sorted_items,100)
    return list(keywords.keys())

In [72]:
def analyze_top_n_neg_bigram(negative_texts):
    neg_tf_idf_vector=neg_tfidf_transformer.transform(pos_cv.transform(negative_texts))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(neg_tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(neg_cv.get_feature_names(),sorted_items,100)
    return list(keywords.keys())

### Clustering

In [39]:
import torch
import pandas as pd
from ngram_manager import NgramManager
from gensim.models import KeyedVectors
import numpy as np
from nltk.cluster import KMeansClusterer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import tokenize_uk

In [38]:
class ClusterNgram:
    def __init__(self, word2vec):
        self.word2vec = word2vec
        
        
    
    def cluster_n_grams(self, ngrams, num_cluster):
        X = self.convert_ngrams_to_vectors(ngrams)
        kclusterer = KMeansClusterer(num_cluster, 
                                     distance=nltk.cluster.util.cosine_distance, 
                                     repeats=100,
                                     avoid_empty_clusters=True)
        y = kclusterer.cluster(X, assign_clusters=True)
        
        top_indices = self.find_top_n_gram_indices(X, y, kclusterer.means())
        
        return np.array(ngrams)[top_indices]
        
        
    
    def find_top_n_gram_indices(self, X, y, centroids):
        min_dis = len(centroids)*[100]
        top_indices = len(centroids)*[-1]
        for i in range(0, len(centroids)):
            for j in range(0, len(X)):
                if y[j] != i:
                    continue
                
                dis = nltk.cluster.util.cosine_distance(X[j], centroids[i])
                if min_dis[i] > dis:
                    min_dis[i] = dis
                    top_indices[i] = j
        
        return top_indices
                    
    
    def convert_ngrams_to_vectors(self, ngrams):
        X = []
        for ngram in ngrams:
            try:
                words = tokenize_uk.tokenize_words(ngram)
                x = word2vec[words[0]]
                for i in range(1, len(words)):
                    x = x + word2vec[words[i]]
                X.append(x)
            except Exception as e:
                print(e)
#                 X.append(np.zeros(300))
        return X

In [40]:
embeddings_file = "/home/dbabenko/Downloads/ubercorpus.cased.tokenized.word2vec.300d"

In [41]:
word2vec = KeyedVectors.load_word2vec_format(embeddings_file)

In [52]:
cluster_ngram = ClusterNgram(word2vec)

### Analyze

In [62]:
pos_bigrams = analyze_top_n_pos_bigram(blum_df['pos_text'].values)

In [73]:
neg_bigrams = analyze_top_n_neg_bigram(blum_df['neg_text'].values)

In [56]:
cluster_ngram.cluster_n_grams(pos_bigrams, 7)

array(['кава чай', 'готель центрі', 'чудовий готель', 'сніданок номер',
       'чисто комфортно', 'привітний персонал', 'зручне розташування'],
      dtype='<U22')

In [ ]:
cluster_ngram.cluster_n_grams(pos_bigrams, 7)

In [74]:
cluster_ngram.cluster_n_grams(neg_bigrams, 7)

array(['те готель', 'тепла вода', 'двері номер', 'маленький холодильник',
       'телевізора номері', 'включений вартість', '30 40'], dtype='<U22')

In [65]:
neg_bigrams

['чудовий персонал',
 'чудовий готель',
 'чистота персонал',
 'чистота затишок',
 'чисто тихо',
 'чисто персонал',
 'чисто охайно',
 'чисто комфортно',
 'ціна якість',
 'центрі міста',
 'хороший сніданок',
 'хороший персонал',
 'уважний персонал',
 'тиша спокій',
 'супер розташування',
 'ставлення персоналу',
 'сніданок смачний',
 'сніданок різноманітний',
 'сніданок номер',
 'сніданок вибір',
 'смачні сніданки',
 'свіжим ремонтом',
 'розташування чудове',
 'розташування чистота',
 'розташування персонал',
 'розташування готелю',
 'розташування відмінне',
 'розташований центрі',
 'прямо центрі',
 'приємний персонал',
 'привітний персонал',
 'привабливий персонал',
 'питна вода',
 'персонал чистота',
 'персонал уважний',
 'персонал ввічливий',
 'номері чисто',
 'місце розташування',
 'кава чай',
 'зупинятися готелі',
 'зручно чисто',
 'зручне розташування',
 'затишний номер',
 'затишне місце',
 'доброзичливий персонал',
 'готель чистий',
 'готель центрі',
 'готель центру',
 'готель має'

In [71]:
import pickle

In [115]:
pickle.dump(pos_tfidf_transformer, open("pos_tfidf_transformer.pickle", "wb"))

In [116]:
pickle.dump(pos_cv, open("pos_cv.pickle", "wb"))

In [ ]:
# open a file, where you stored the pickled data
file = open('important', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()

In [ ]:
pos_cv

In [117]:
neg_text

NameError: name 'neg_text' is not defined

In [75]:
pos_texts[123]

'Хороше розташування. Безпечна парковка на території готелю. Наявність холодильника та чайника в номері.\nЩоденне прибирання.'

In [119]:
neg_texts[123]

'Головне не переплутати з іншим готелем "Верховина", котрий знаходиться на відстані 2,5 км. При поселенні потрібно пройти квест "знайди рецепцію, котра знаходиться в корпусі №Х, а потім звідти іди на поселення в корпус №У"'

## Read dataset and create docs

In [9]:
full_df['title'].iloc[0]

'Лише дівчата на рецепції - три рази мені міняли кімнату.'

In [12]:
title = literal_eval(full_df.iloc[0]['title'])

SyntaxError: invalid syntax (<unknown>, line 1)

In [10]:
' '.join(word for word in title)

NameError: name 'title' is not defined

In [15]:
def create_docs(data):
    docs = []
    for i in range(0, len(data)):
        text = ""
        
        title = data.iloc[i]['title']
        if len(title) > 0 and title[0] != 'Nan':
            text += title
            
        pos = data.iloc[i]['pos_text']
        if len(pos) > 0 and pos[0] != 'Nan':
            text += ' '
            text += pos
            
        neg = data.iloc[i]['neg_text']
        if len(neg) > 0 and neg[0] != 'Nan':
            text += ' '
            text += neg
            
        docs.append(text)
    return docs
        

In [16]:
docs = create_docs(full_df)

### Create the IDF

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
stop_words = read_stop_words('data/ukrainian-stopwords.txt')

In [22]:
cv=CountVectorizer(max_df=0.9,stop_words=stop_words)
word_count_vector=cv.fit_transform(docs)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [23]:
word_count_vector.shape

(134083, 91583)

Let's limit our vocabulary size to 10,000

In [24]:
cv=CountVectorizer(max_df=0.85,stop_words=stop_words,max_features=1000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(134083, 1000)

### TfidfTransformer to Compute Inverse Document Frequency (IDF) 

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

Let's look at some of the IDF values:

In [26]:
tfidf_transformer.idf_

array([4.88795706, 4.41686193, 5.77095315, 6.0609854 , 5.69715961,
       6.02289655, 4.98057702, 5.14846648, 6.34007702, 6.67499526,
       6.56789712, 5.2843625 , 3.19101645, 6.35417279, 6.04812724,
       7.42632439, 6.32464462, 5.10050892, 6.75884957, 1.95952893,
       4.38884889, 7.27283226, 5.10005878, 5.83455314, 5.23629409,
       5.67292579, 4.9254174 , 6.00716589, 6.52022365, 6.48905706,
       6.63035448, 6.08479605, 6.8555792 , 5.65630491, 5.50439941,
       6.7059028 , 5.72535385, 6.51465261, 5.65630491, 6.29447642,
       6.51650618, 6.59161365, 5.33201694, 6.59361565, 6.48186279,
       5.74403012, 6.62413684, 5.99277715, 6.24377765, 5.97319002,
       5.45442188, 5.95503682, 5.84303176, 6.35259675, 6.54664028,
       6.30944676, 5.93098966, 6.08359195, 5.97426818, 6.38459948,
       5.97426818, 5.66894331, 6.03543232, 5.40011837, 6.23254158,
       6.57181102, 5.78335366, 6.6663372 , 6.63243564, 6.55817887,
       4.83375573, 6.24377765, 4.44802429, 5.6361022 , 5.99939

### Computing TF-IDF and Extracting Keywords

In [27]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [34]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n===Review text===")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


===Review text===
Лише дівчата на рецепції - три рази мені міняли кімнату. Лише дівчата на рецепції - три рази мені міняли кімнату. Дякую їм. Все. Одного досвіду вистарчило, щоб більше сюди не повертатися. Готель лише для гостей, яким байдуже умови , чистота і комфорт.

===Keywords===
рази 0.472
дівчата 0.458
кімнату 0.442
рецепції 0.4
сюди 0.221
умови 0.218
гостей 0.207
комфорт 0.204
чистота 0.145
готель 0.098


In [29]:
a = tf_idf_vector=tfidf_transformer.transform(cv.transform(docs[:1]))

In [30]:
cv.transform(docs[:2])

<2x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [31]:
sorted_items=sort_coo(a.tocoo())

In [32]:
sorted_items

[(714, 0.47177032223963566),
 (283, 0.4583165725717789),
 (422, 0.4423654769281214),
 (739, 0.39990782054595764),
 (838, 0.22144445972314214),
 (875, 0.2180827599072353),
 (210, 0.20699220385566333),
 (390, 0.20439509883082757),
 (936, 0.1447041801699384),
 (215, 0.09800527891767899)]